## Identifying last data import period

In DHIS2, a **data value** is uniquely identified by the composite key (**orgUnit, dataElement, period, categoryOptionCombo, attributeOptionCombo**).
The **period** is what tells you which time slice the data belongs to.
The **created** and **lastupdated** timestamps only tell you *when the record was inserted or changed* — they do not indicate the reporting period.

Example: If you import January 2024 monthly data today, the data value’s period is `202401` (Jan 2024). The `created` and `lastupdated` timestamps will show today’s date, but the reporting period remains January 2024 as shown below.

```python
{
      "dataElement": "<data_element_uid>",
      "period": "202401",
      "orgUnit": "<org_unit_uid>",
      "value": "1",
      "storedBy": "admin",
      "created": "2025-10-08",
      "lastUpdated": "2025-10-08"
}
```

Therefore, to determine the **last data import period**, you must look at the **period** dimension, not timestamps. `dhis2-python-client` provides a convenience method that finds the most recent **period** (by period chronology) for which data exists, scoped by a data element and an organisational level. See below an example

In [3]:
import json
from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Client configuration
cfg = ClientSettings(
  base_url="http://localhost:8080",
  username="admin",
  password="district")

client = DHIS2Client(settings=cfg)

data_element_uid = 'Fyul5papro0'
level = 2
import_period_info = client.analytics_latest_period_for_level(data_element_uid, level)

print(json.dumps(import_period_info, indent=4))

{
    "meta": {
        "dataElement": "Fyul5papro0",
        "level": 2,
        "periodType": "MONTHLY",
        "calendar": "iso8601",
        "years_checked": 2
    },
    "existing": {
        "id": "202405",
        "startDate": "2024-05-01",
        "endDate": "2024-05-31"
    },
    "next": {
        "id": "202406",
        "startDate": "2024-06-01",
        "endDate": "2024-06-30"
    }
}


The response tells us, this data element is collected **Monthly**. The latest period with data is **May 2025 (202505)**. The next expected period is **June 2025 (202506)**.